In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
sns.set_style('darkgrid')
mpl.rcParams['figure.figsize'] = [18,10]

In [17]:
non_numeric = ['BMI_class', 'Height_class', 'Gender', 'Component', 'Branch']

def load_ansur(cols_to_drop):
    df_m = pd.read_csv('data/ANSUR_II_MALE.csv')
    df_f = pd.read_csv('data/ANSUR_II_FEMALE.csv')
    ansur_df = pd.concat([df_m, df_f], axis=0)

    X = ansur_df.drop(non_numeric, axis=1)
    y = ansur_df['Gender']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    return X, y, X_train, X_test, y_train, y_test

In [18]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

### Creating a logistic regression model

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression()
lr.fit(X_train_std, y_train)

X_test_std = scaler.transform(X_test)

In [20]:
y_pred = lr.predict(X_test_std)
print(accuracy_score(y_test, y_pred))

1.0


### Inspecting the feature coefficients 

In [21]:
print(lr.coef_[0][:10])

[ 0.16538097  0.11415374 -0.11988966 -0.35318174 -0.10102145 -0.12958967
  0.29849897  0.62395995  0.78708997 -0.71298706]


In [22]:
coef_dict = dict(zip(X.columns, abs(lr.coef_[0])))

{k: v for i, (k, v) in enumerate(coef_dict.items()) if i < 10}

{'abdominalextensiondepthsitting': 0.16538096650652276,
 'acromialheight': 0.114153743902519,
 'acromionradialelength': 0.11988966102355153,
 'anklecircumference': 0.35318174207131736,
 'axillaheight': 0.10102145271792597,
 'balloffootcircumference': 0.12958967443975758,
 'balloffootlength': 0.29849897180764645,
 'biacromialbreadth': 0.6239599493764185,
 'bicepscircumferenceflexed': 0.7870899693843955,
 'bicristalbreadth': 0.7129870615083279}

In [23]:
low_coef = {k: v for k, v in coef_dict.items() if v < .401}

cols = [k for k, v in low_coef.items()]

In [24]:
X.drop(cols, axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

lr.fit(scaler.fit_transform(X_train), y_train)

print(accuracy_score(y_test, lr.predict(scaler.transform(X_test))))

1.0


In [26]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [27]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=LogisticRegression(), n_features_to_select=5, verbose=0)
rfe.fit(X_train_std, y_train)

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='auto', n_jobs=None, penalty='l2',
                                 random_state=None, solver='lbfgs', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=5, step=1, verbose=0)

In [28]:
X.columns[rfe.support_]

Index(['chestdepth', 'hipbreadthsitting', 'lateralmalleolusheight',
       'neckcircumference', 'shouldercircumference'],
      dtype='object')

In [29]:
#print(dict(zip(X.columns, rfe.ranking_)))

In [30]:
print(accuracy_score(y_test, rfe.predict(X_test_std)))

0.9950576606260296


In [35]:
def load_pima(cols_to_drop):
    df = pd.read_csv('data/PimaIndians.csv')

    X = df.drop(cols_to_drop, axis=1)
    y = df['test']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    return X, y, X_train, X_test, y_train, y_test

X, y, X_train, X_test, y_train, y_test = load_pima('test')

scaler = StandardScaler()
lr = LogisticRegression()

In [36]:
# Fit the scaler on the training features and transform these in one go
X_train_std = scaler.fit_transform(X_train)

# Fit the logistic regression model on the scaled training data
lr.fit(X_train_std, y_train)

# Scale the test features
X_test_std = scaler.transform(X_test)

# Predict diabetes presence on the scaled test set
y_pred = lr.predict(X_test_std)

# Prints accuracy metrics and feature coefficients
print("{0:.1%} accuracy on test set.".format(accuracy_score(y_test, y_pred))) 
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

78.8% accuracy on test set.
{'pregnant': 0.17, 'glucose': 1.24, 'diastolic': 0.05, 'triceps': 0.05, 'insulin': 0.03, 'bmi': 0.6, 'family': 0.64, 'age': 0.42}


In [38]:
X, y, X_train, X_test, y_train, y_test = load_pima('test')

X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# Create the RFE with a LogisticRegression estimator and 3 features to select
rfe = RFE(estimator=LogisticRegression(), n_features_to_select=3, verbose=1)

# Fits the eliminator to the data
rfe.fit(X_train_std, y_train)

# Print the features and their ranking (high = dropped early on)
print(dict(zip(X.columns, rfe.ranking_)))

# Print the features that are not eliminated
print(X.columns[rfe.support_])

# Calculates the test set accuracy
acc = accuracy_score(y_test, rfe.predict(X_test_std))
print("{0:.1%} accuracy on test set.".format(acc)) 

Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
{'pregnant': 2, 'glucose': 1, 'diastolic': 5, 'triceps': 4, 'insulin': 6, 'bmi': 1, 'family': 3, 'age': 1}
Index(['glucose', 'bmi', 'age'], dtype='object')
81.4% accuracy on test set.


In [41]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric)

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()

rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [43]:
print(accuracy_score(y_test, rf.predict(X_test)))

0.9928610653487095


In [44]:
print(rf.feature_importances_)

[0.00206631 0.00085132 0.00037488 0.00099212 0.0009641  0.01011892
 0.00136837 0.07127259 0.00220716 0.00504109 0.02323911 0.02559065
 0.00066058 0.00582394 0.00151748 0.01573475 0.00315212 0.0008796
 0.00213329 0.00108655 0.00277378 0.00654128 0.00074444 0.00149168
 0.00697598 0.01908575 0.00060196 0.00652669 0.00057112 0.0010713
 0.00090475 0.00235291 0.00032742 0.01524344 0.00487481 0.01896129
 0.00138604 0.03525249 0.00536859 0.00139054 0.00063838 0.05699923
 0.06933629 0.0008951  0.00104036 0.00117986 0.00036979 0.02360453
 0.00034314 0.01425212 0.02788376 0.00105118 0.00033213 0.00283422
 0.00643822 0.00041519 0.00078931 0.00135627 0.01906717 0.00481514
 0.00248674 0.16255435 0.08284147 0.00557559 0.00035937 0.00639895
 0.00174778 0.05067019 0.00737041 0.00115761 0.00544367 0.02082252
 0.00149705 0.00085596 0.00481622 0.00134939 0.00671688 0.00141185
 0.00063185 0.0006193  0.00055278 0.00098847 0.00083951 0.00882373
 0.00096772 0.00210244 0.00125869 0.00062147 0.00042504 0.065709

In [45]:
mask = rf.feature_importances_ > 0.03
print(mask)

[False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False  True False False False  True  True False False False False False
 False False False False False False False False False False False False
 False  True  True False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False  True False False False False]


In [46]:
X_reduced = X_train.loc[:, mask]
print(X_reduced.columns)

Index(['biacromialbreadth', 'forearmcircumferenceflexed', 'handbreadth',
       'handcircumference', 'neckcircumference', 'neckcircumferencebase',
       'shouldercircumference', 'wristcircumference'],
      dtype='object')


### RFE with random forests

In [47]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=RandomForestClassifier(), 
          n_features_to_select=6, step=10, 
            verbose=1)

rfe.fit(X_train, y_train)

Fitting estimator with 94 features.
Fitting estimator with 84 features.
Fitting estimator with 74 features.
Fitting estimator with 64 features.
Fitting estimator with 54 features.
Fitting estimator with 44 features.
Fitting estimator with 34 features.
Fitting estimator with 24 features.
Fitting estimator with 14 features.


RFE(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                     class_weight=None, criterion='gini',
                                     max_depth=None, max_features='auto',
                                     max_leaf_nodes=None, max_samples=None,
                                     min_impurity_decrease=0.0,
                                     min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,
                                     min_weight_fraction_leaf=0.0,
                                     n_estimators=100, n_jobs=None,
                                     oob_score=False, random_state=None,
                                     verbose=0, warm_start=False),
    n_features_to_select=6, step=10, verbose=1)

In [48]:
print(X_train.columns[rfe.support_])

Index(['biacromialbreadth', 'handbreadth', 'handcircumference',
       'neckcircumference', 'neckcircumferencebase', 'shouldercircumference'],
      dtype='object')


In [50]:
X, y, X_train, X_test, y_train, y_test = load_pima('test')

In [51]:
# Fit the random forest model to the training data
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

# Calculate the accuracy
acc = accuracy_score(y_test, rf.predict(X_test))

# Print the importances per feature
print(dict(zip(X_train.columns, rf.feature_importances_.round(2))))

# Print accuracy
print("{0:.1%} accuracy on test set.".format(acc))

{'pregnant': 0.07, 'glucose': 0.23, 'diastolic': 0.08, 'triceps': 0.07, 'insulin': 0.16, 'bmi': 0.12, 'family': 0.12, 'age': 0.16}
72.9% accuracy on test set.


In [52]:
# Create a mask for features importances above the threshold
mask = rf.feature_importances_ > 0.15

# Prints out the mask
print(mask)

[False  True False False  True False False  True]


In [53]:
# Create a mask for features importances above the threshold
mask = rf.feature_importances_ > 0.15

# Apply the mask to the feature dataset X
reduced_X = X_train.loc[:, mask]

# prints out the selected column names
print(reduced_X.columns)

Index(['glucose', 'insulin', 'age'], dtype='object')


In [54]:
# Wrap the feature eliminator around the random forest model
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=2, verbose=1)

In [55]:
# Fit the model to the training data
rfe.fit(X_train, y_train)

Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.


RFE(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                     class_weight=None, criterion='gini',
                                     max_depth=None, max_features='auto',
                                     max_leaf_nodes=None, max_samples=None,
                                     min_impurity_decrease=0.0,
                                     min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,
                                     min_weight_fraction_leaf=0.0,
                                     n_estimators=100, n_jobs=None,
                                     oob_score=False, random_state=None,
                                     verbose=0, warm_start=False),
    n_features_to_select=2, step=1, verbose=1)

In [ ]:
!../gitbsh